In [4]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

%matplotlib inline
#%matplotlib notebook
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets



 # Very Simple Epidemic 
 
 
 * $\color{blue} S$ - susceptible
 * $\color{red} I$ - infective
 
 
 
 Population is closed: $S(t)+I(t)=N,$ $N = \it\mbox{const.}$
 
 $\beta$ is effective contact rate, contact rate $\times$ transmissibility
 
 
 
  $$
 \left\{
\begin{array}\\
  \dfrac{dS}{dt} = -\beta  S \, \dfrac{ I}{ N} % \quad \mbox{ change in } S \mbox{ population over time}\  
  \\
    \dfrac{dI}{dt}  = \, \beta S \, \dfrac{ I}{ N} % \quad \mbox{ change in } I \mbox{ population over time}
  \end{array}
  \right., % \quad   \left(\dfrac{dS}{dt}+\dfrac{dI}{dt} =  0  \right)
  \quad t \in [0,T_{end}]
  $$
  
 

  $$ \dfrac{dI}{dt} = \beta \left(N-I\right)  \dfrac{ I}{ N} , \quad  S(t) =N- I(t)   $$  

### Steady states: 


$$ \dfrac{dI}{dt}=0
\quad \Longleftrightarrow \quad
\beta I \left(1-\dfrac{ I}{ N}\right) = 0 \quad \Longleftrightarrow \quad
\begin{array}\\
I =0\\
I =N 
\end{array}
$$

Meaning, if we  ever reach an unchaning state, it is either these two options: 

(i) everyone is infected, or 

(ii) none is infected.



### Example

$N=10 \,000$ people

$T_{end} = 160$ days 
 


In [5]:
N=10000
days = range(0, 160)

mylayout =Layout(width='10cm')

# The SI model 
def derivI(state, t, N, beta):
    I = state
    dIdt = beta * (N-I) * I / N 
    return dIdt



def g(I0,r):
    #r=effective_contact_rate
    susceptible = N - I0
    ret = odeint(derivI, I0,days,
             args=(N, r))
    
    I = ret.T[0,:]
    df = pd.DataFrame({
        'suseptible': N - I,
        'infective': I,
         'day': days})
    
    df.plot(x='day',
            y=['infective', 'suseptible'],
            color=['#bb6424', '#aac6ca'],
             );
    plt.show()
    dayP='inf';
    if (r>0) and (I0>0):
        dayP = np.ceil(np.log(0.9/0.1*(N-I0)/I0)/r);
        dayP=np.int32(dayP)
    print('days before 90% of the populaton is infected = ', dayP);
    print('number of infective after 10 days = %.2f  ' % I[10]);
    #print('days before 90% of the populaton are infected =',nd()np.argmax((np.round(I)>0.9*total_pop)));
    #return df.round(2);

    
interact(g, I0=widgets.Dropdown(
                options=[0, 1, 100, 1000],
                value=1,
                description='I(0):',
                disabled=False,),
             r=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1, description='beta:',
                                  continuous_update=False,layout=mylayout,readout=True, readout_format='.1f'));
    
    
#     I0=widgets.FloatSlider(min=0, max=1000, step=1, value=1, description='I(0):',
#       continuous_update=False,layout=mylayout,readout=True, readout_format='5d'),
#






interactive(children=(Dropdown(description='I(0):', index=1, options=(0, 1, 100, 1000), value=1), FloatSlider(…

### Question 1: 
Assume the exponential growth $$I(t) = I(0)\, e^{\beta t},$$ 

and calculate the number of infected after 10 days. Compare with the result above!

In [6]:
1*np.exp(0.1*10)

2.718281828459045

# Simple epidemic (SIS)
 
 
 * $\color{blue} S$ - suceptible
 * $\color{red} I$ - infective
 
 
 
 Population is closed: $S(t)+I(t)=N,$ $N = \it\mbox{const.}$
 
 $\beta$ is effective contact rate (contact rate $\times$ transmissibility)
 
 $\gamma$ is recovery rate 
 
 
 
  $$
 \left\{
\begin{array}\\
  \dfrac{dS}{dt} = -\beta  S \, \dfrac{ I}{ N} + \gamma I% \quad \mbox{ change in } S \mbox{ population over time}\  
  \\
    \dfrac{dI}{dt}  = \, \beta S \, \dfrac{ I}{ N} - \gamma I % \quad \mbox{ change in } I \mbox{ population over time}
  \end{array}
  \right., % \quad   \left(\dfrac{dS}{dt}+\dfrac{dI}{dt} =  0  \right)
  \quad t \in [0, T_{end}]
  $$
  
 

  $$ \dfrac{dI}{dt} = \beta I \left(1 - \frac{\gamma} {\beta} -\dfrac{ I}{ N}\right ), \quad  S(t) =N- I(t)  $$


### Steady states: 


$$
 \dfrac{dI}{dt}=0
\quad \Longleftrightarrow \quad 
\beta I \left(1 - \frac{\gamma} {\beta} -\dfrac{ I}{ N}\right )=0  \quad \Longleftrightarrow \quad
\left\{
\begin{array}\\
I =0 \\ 
I =\left(1 - \frac{\gamma} {\beta} \right)N, \mbox{ if } \beta>\gamma 
\end{array}
\right.
$$

Introduce the basic reproduction number $R_0 = \beta\,/\gamma$ (contact rate $\times$ transmissibility $/$ recovery rate) 

Meaning, if we ever reach an unchaning state, it is either these two options: 

(i)  everyone recovers, or 

(ii) the infection persists.

### Example

$N=10 \,000,$  $Days = 160,$ $\gamma = 1/4$

In [7]:
# The SIS model 

gamma=1/4;
def derivSIS(state, t, N, beta, gamma):
    I = state
    dIdt = beta * (N-I) * I / N - gamma*I 
    return dIdt


def g(I0,gamma,R0):
    beta=R0*gamma;
    #beta=effective_contact_rate
    ret = odeint(derivSIS, I0,days,
             args=(N, beta,gamma))
    
    I = ret.T[0,:]
    df = pd.DataFrame({
        'suseptible': N - I,
        'infective': I,
         'day': days})
    
    df.plot(x='day',
            y=['infective', 'suseptible'],
            color=['#bb6424', '#aac6ca'],
             );
    plt.show()
    dayP='inf';
#         dayP = np.ceil(np.log(0.9/0.1*(N-I0)/I0)/r);
#         dayP=np.int32(dayP)
#     print('days before 90% of the populaton is infected = ', dayP);
    print('Steady states: I=0, I=',(1-gamma/beta)*N)
    print('number of infected after 10 days = %.2f  ' % I[10]);
    #print('days before 90% of the populaton are infected =',nd()np.argmax((np.round(I)>0.9*total_pop)));
    #return df.round(2);

    
interact(g, I0=widgets.Dropdown(
                options=[0, 1, 1000, 1500],
                value=1500,
                description='I(0):',
                disabled=True,),
            gamma=widgets.Dropdown(
                options=[0.1, 0.25],
                value=0.25,
                description='gamma:',
                disabled=True,),
             R0=widgets.FloatSlider(min=0.6, max=3., step=0.2, value=2., description='R0:',
                                  continuous_update=False,layout=mylayout,readout=True, readout_format='.1f'));
    
    
#     I0=widgets.FloatSlider(min=0, max=1000, step=1, value=1, description='I(0):',
#       continuous_update=False,layout=mylayout,readout=True, readout_format='5d'),
#




interactive(children=(Dropdown(description='I(0):', disabled=True, index=3, options=(0, 1, 1000, 1500), value=…

### Question 2: 
Remember that $R_0 = \beta\, /\,\gamma.$ Assume that the recovery rate $\gamma = 0.1.$ 

What must be the effective contact rate $\beta$ to completely erradicate the epidemics?

### Question 3: 
What are the strategies to keep $R_0$ below 1?



# Simple epidemic (SIR)
 
 
 * $\color{blue} S$ - suceptible
 * $\color{red} I$ - infective
 * $\color{green} R$ - removed (dead or alive!)
 
 
 
 Population is closed: $S(t)+I(t)+R(t)=N,$ $N = \it\mbox{const.}$
 
 $\beta$ is effective contact rate (contact rate $\times$ transmissibility)
 
 $\gamma$ is recovery rate 
 
 
 
  $$
 \left\{
\begin{array}\\
  \dfrac{dS}{dt} = -\beta  S \, \dfrac{ I}{ N} % \quad \mbox{ change in } S \mbox{ population over time}\  
  \\
    \dfrac{dI}{dt}  = \, \beta S \, \dfrac{ I}{ N} - \gamma I % \quad \mbox{ change in } I \mbox{ population over time}
    \\
    \dfrac{dR}{dt}  = \gamma I\\
  \end{array}
  \right., % \quad   \left(\dfrac{dS}{dt}+\dfrac{dI}{dt} =  0  \right)
  \quad t \in [0, T_{end}]
  $$
   
   
   $$
 \left\{
\begin{array}\\
  \dfrac{dS}{dt} = -\beta  S \, \dfrac{ I}{ N} % \quad \mbox{ change in } S \mbox{ population over time}\  
  \\
    \dfrac{dI}{dt}  = \, \beta S \, \dfrac{ I}{ N} - \gamma I % \quad \mbox{ change in } I \mbox{ population over time}
    \end{array}
  \right., % \quad   \left(\dfrac{dS}{dt}+\dfrac{dI}{dt} =  0  \right)
  \quad t \in [0, T_{end}]
  $$ 

  $$
  R(t)  = N-S(t) -  I(t). \quad \quad \quad \quad
  $$


### Steady states: 


$$
I=0 \quad \mbox{and} \quad S+R=N
$$

Meaning, the infection always dies out, but look at the progression!



In [8]:
# The SIR model differential equations.

N=10000
gamma=1/4;
days = range(0, 160)

def deriv(state, t, N, beta, gamma):
    S, I, R = state
    # Change in S population over time
    dSdt = -beta * S * I / N
    # Change in I population over time
    dIdt = beta * S * I / N - gamma * I
    # Change in R population over time
    dRdt = gamma * I
    return dSdt, dIdt, dRdt


mylayout =Layout(width='10cm')

def g(I0,gamma,R0):
    #r=effective_contact_rate
    beta=R0*gamma;
    S0 = N - I0
    ret = odeint(deriv, [S0, I0, 0],
             days,
             args=(N, beta, gamma))
    
    S, I, R = ret.T
    
    df = pd.DataFrame({
    'suseptible': S,
    'infective': I,
    'recovered': R,
    'day': days})
    
    df.plot(x='day',
            y=['infective', 'suseptible', 'recovered'],
            color=['#bb6424', '#aac6ca', '#cc8ac0'],
             );
    plt.show()
    
    print('max number of infected =',np.max(I));

 
interact(g, I0=widgets.Dropdown(
                options=[0, 1, 1000, 1500],
                value=1,
                description='I(0):',
                disabled=True,),
            gamma=widgets.Dropdown(
                options=[0.1, 0.25],
                value=0.25,
                description='gamma:',
                disabled=True,),
             R0=widgets.FloatSlider(min=0.8, max=4., step=0.1, value=2., description='R0:',
                                  continuous_update=False,layout=mylayout,readout=True, readout_format='.1f'));
    
    

# interact(g, r=widgets.FloatSlider(min=0, max=5, step=0.1, value=0.5, description='eff.contact rate:',
#                                   continuous_update=False,layout=mylayout,readout=True, readout_format='.1f',),
#             b=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25, description='recovery rate:',
#                                   continuous_update=False,layout=mylayout,readout=True, readout_format='.2f',));
    



# If you get the error:
#
#     When stacked is True, each column must be either all
#     positive or negative.infected contains both...
#
# just change stacked=True to stacked=False

interactive(children=(Dropdown(description='I(0):', disabled=True, index=1, options=(0, 1, 1000, 1500), value=…

### Question 4: 
Let the recovery rate $\gamma =0.25$ as above.

What value of $R_0>1$ will keep the maximum number of infective below $1600$?

Calculate the corresponding $\beta.$



In [9]:
R0 = 2.0 # your answer, then Ctrl+Enter
beta= R0*gamma

print(beta)

0.5


### Question 5: 

Use $\beta$ from Question 4, and remember that 
$$\beta= \mbox{ contact rate } \times \mbox{ transmissibility }$$
 
Assume that $\mbox{transmissibility} = 0.1 $


How many people on average one can meet per day?



In [10]:
contact_rate =0 # insert your answer, then Ctrl+Enter

print(contact_rate,'people per day')

0 people per day
